# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 23 2023 8:30AM,257978,10,MSP218809,"Methapharm, Inc.",Released
1,Feb 23 2023 8:30AM,257980,10,MSP218819,"Methapharm, Inc.",Released
2,Feb 23 2023 8:30AM,257980,10,MSP218820,"Methapharm, Inc.",Released
3,Feb 23 2023 8:30AM,257980,10,MSP218821,"Methapharm, Inc.",Released
4,Feb 23 2023 8:30AM,257980,10,MSP218808,"Methapharm, Inc.",Released
5,Feb 23 2023 8:30AM,257980,10,MSP218810,"Methapharm, Inc.",Released
6,Feb 23 2023 8:30AM,257980,10,MSP218812,"Methapharm, Inc.",Released
7,Feb 23 2023 8:30AM,257980,10,MSP218813,"Methapharm, Inc.",Released
8,Feb 23 2023 8:30AM,257980,10,MSP218816,"Methapharm, Inc.",Released
9,Feb 23 2023 8:30AM,257980,10,MSP218817,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
32,257977,Released,1
33,257978,Released,6
34,257979,Released,1
35,257980,Released,10
36,257981,Released,9


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
257977,NaN,NaN,1.0
257978,NaN,NaN,6.0
257979,NaN,NaN,1.0
257980,NaN,NaN,10.0
257981,NaN,NaN,9.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257892,0.0,1.0,0.0
257910,0.0,1.0,1.0
257914,0.0,7.0,12.0
257916,0.0,0.0,1.0
257925,0.0,1.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257892,0,1,0
257910,0,1,1
257914,0,7,12
257916,0,0,1
257925,0,1,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257892,0,1,0
1,257910,0,1,1
2,257914,0,7,12
3,257916,0,0,1
4,257925,0,1,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257892,,1,
1,257910,,1,1
2,257914,,7,12
3,257916,,,1
4,257925,,1,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 23 2023 8:30AM,257978,10,"Methapharm, Inc."
1,Feb 23 2023 8:30AM,257980,10,"Methapharm, Inc."
14,Feb 23 2023 8:30AM,257979,10,"Methapharm, Inc."
16,Feb 23 2023 8:30AM,257978,10,Methapharm-G
17,Feb 22 2023 3:13PM,257981,10,ISDIN Corporation
26,Feb 22 2023 2:44PM,257977,10,"Senseonics, Incorporated"
27,Feb 22 2023 2:43PM,257976,10,Emerginnova
31,Feb 22 2023 2:33PM,257975,19,ACG North America LLC
32,Feb 22 2023 2:24PM,257974,18,"Emersa Waterbox, LLC"
33,Feb 22 2023 1:42PM,257969,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 23 2023 8:30AM,257978,10,"Methapharm, Inc.",,,6
1,Feb 23 2023 8:30AM,257978,10,Methapharm-G,,,6
2,Feb 23 2023 8:30AM,257980,10,"Methapharm, Inc.",,,10
3,Feb 23 2023 8:30AM,257979,10,"Methapharm, Inc.",,,1
4,Feb 22 2023 3:13PM,257981,10,ISDIN Corporation,,,9
5,Feb 22 2023 2:44PM,257977,10,"Senseonics, Incorporated",,,1
6,Feb 22 2023 2:43PM,257976,10,Emerginnova,,,4
7,Feb 22 2023 2:33PM,257975,19,ACG North America LLC,,,1
8,Feb 22 2023 2:24PM,257974,18,"Emersa Waterbox, LLC",,,1
9,Feb 22 2023 1:42PM,257969,10,ISDIN Corporation,,,64


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 23 2023 8:30AM,257978,10,"Methapharm, Inc.",6,,
1,Feb 23 2023 8:30AM,257978,10,Methapharm-G,6,,
2,Feb 23 2023 8:30AM,257980,10,"Methapharm, Inc.",10,,
3,Feb 23 2023 8:30AM,257979,10,"Methapharm, Inc.",1,,
4,Feb 22 2023 3:13PM,257981,10,ISDIN Corporation,9,,
5,Feb 22 2023 2:44PM,257977,10,"Senseonics, Incorporated",1,,
6,Feb 22 2023 2:43PM,257976,10,Emerginnova,4,,
7,Feb 22 2023 2:33PM,257975,19,ACG North America LLC,1,,
8,Feb 22 2023 2:24PM,257974,18,"Emersa Waterbox, LLC",1,,
9,Feb 22 2023 1:42PM,257969,10,ISDIN Corporation,64,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 23 2023 8:30AM,257978,10,"Methapharm, Inc.",6,,
1,Feb 23 2023 8:30AM,257978,10,Methapharm-G,6,,
2,Feb 23 2023 8:30AM,257980,10,"Methapharm, Inc.",10,,
3,Feb 23 2023 8:30AM,257979,10,"Methapharm, Inc.",1,,
4,Feb 22 2023 3:13PM,257981,10,ISDIN Corporation,9,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 23 2023 8:30AM,257978,10,"Methapharm, Inc.",6.0,NaN,NaN
1,Feb 23 2023 8:30AM,257978,10,Methapharm-G,6.0,NaN,NaN
2,Feb 23 2023 8:30AM,257980,10,"Methapharm, Inc.",10.0,NaN,NaN
3,Feb 23 2023 8:30AM,257979,10,"Methapharm, Inc.",1.0,NaN,NaN
4,Feb 22 2023 3:13PM,257981,10,ISDIN Corporation,9.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 23 2023 8:30AM,257978,10,"Methapharm, Inc.",6.0,0.0,0.0
1,Feb 23 2023 8:30AM,257978,10,Methapharm-G,6.0,0.0,0.0
2,Feb 23 2023 8:30AM,257980,10,"Methapharm, Inc.",10.0,0.0,0.0
3,Feb 23 2023 8:30AM,257979,10,"Methapharm, Inc.",1.0,0.0,0.0
4,Feb 22 2023 3:13PM,257981,10,ISDIN Corporation,9.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5674992,226.0,23.0,0.0
16,515913,2.0,0.0,0.0
18,257974,1.0,0.0,0.0
19,1547727,4.0,5.0,1.0
21,515891,2.0,0.0,0.0
22,257916,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5674992,226.0,23.0,0.0
1,16,515913,2.0,0.0,0.0
2,18,257974,1.0,0.0,0.0
3,19,1547727,4.0,5.0,1.0
4,21,515891,2.0,0.0,0.0
5,22,257916,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,226.0,23.0,0.0
1,16,2.0,0.0,0.0
2,18,1.0,0.0,0.0
3,19,4.0,5.0,1.0
4,21,2.0,0.0,0.0
5,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,226.0
1,16,Released,2.0
2,18,Released,1.0
3,19,Released,4.0
4,21,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,18,19,21,22
Status,,,,,,
Completed,0.0,0.0,0.0,1.0,0.0,0.0
Executing,23.0,0.0,0.0,5.0,0.0,0.0
Released,226.0,2.0,1.0,4.0,2.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,18,19,21,22
0,Completed,0.0,0.0,0.0,1.0,0.0,0.0
1,Executing,23.0,0.0,0.0,5.0,0.0,0.0
2,Released,226.0,2.0,1.0,4.0,2.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,18,19,21,22
0,Completed,0.0,0.0,0.0,1.0,0.0,0.0
1,Executing,23.0,0.0,0.0,5.0,0.0,0.0
2,Released,226.0,2.0,1.0,4.0,2.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()